In [1]:
import urllib
from bs4 import BeautifulSoup
import os
import pickle
import re
# from queue import Queue
from threading import Thread
import time
import multiprocessing
from multiprocessing import Process, Queue, current_process, freeze_support

In [2]:
def in_val_set(i):
    return i % 10 == 0

def in_test_set(i):
    return i % 10 in [1, 2]

def in_train_set(i):
    return i % 10 > 2

In [ ]:
start_phrases = ["years ago", "min read", "(reuters) - ", "min read*"]
end_phrases = ["our standards:the thomson reuters trust principles", "reporting by", "editing by", "reporting and writing by", "additional reporting", "writing by"]

bad = 0

# https://stackoverflow.com/questions/3472515/python-urllib2-urlopen-is-slow-need-a-better-way-to-read-several-urls

# Fetch
def fetch_parallel(articles):
    dset = Queue()
    print("started making threads")
    threads = [Thread(target=read_webpage, args = (article, dset)) for article in articles]
    print("finished making threads")
    for t in threads:
        t.start()
    for t in threads:
        t.join()
    return dset

def fetch_parallel2(articles):
    dset = Queue()
    threads = []
    for article in articles:
        thread = Thread(target=read_webpage, args = (article, dset))
        threads.append(thread)
        thread.start()
    for t in threads:
        t.join()
    return dset

def fetch_sequential(articles):
    for article in articles:
        dset = Queue()
        read_webpage(article, dset)


def read_webpage(article, queue):
    print("Starting one!")
    title = article["title"]
    if title[:6] == "TABLE-":
        return

    link = article["href"]
    start_index = -1
    try:
        print("        reading " + link)
        with urllib.request.urlopen(link) as webpage:
            print("        Read it raw")
            content = BeautifulSoup(webpage.read(), "html.parser")
            for unnecessary_content in content(["script", "style", "table"]):
                unnecessary_content.extract()    
            text = content.get_text()
            for phrase in start_phrases:
                if phrase in text.lower():
                    phrase_index = text.lower().index(phrase) + len(phrase)
                    start_index = max(start_index, phrase_index)
            end_index = 99999999
            for phrase in end_phrases:
                if phrase in text.lower():
                    phrase_index = text.lower().index(phrase)
                    if text[phrase_index - 1] == "(":
                        phrase_index -= 1
                    end_index = min(end_index, phrase_index)
            text = text[start_index: end_index]
            text = text.replace("\n", " ")
            title = title.replace("\n", " ")
            queue.put((title, text))
            print("     Finished parsing succeeded")
    except Exception as e:
        print("        ERROR", e)
        global bad
        bad += 1
    
def generate_dset(name, condition):
    # Load data
    articles = []
    for ls in os.listdir('Reuters-full-data-set-master/data'):
#         print("New set", ls)
        if ls.endswith('.pkl'):
            with open('Reuters-full-data-set-master/data/' + ls, 'rb') as f:
                data = pickle.load(f, encoding='latin1')
                new_articles = [article for index, article in enumerate(data) if condition(index)]
                articles += new_articles
#                 print("total article count:", len(articles))
#                 print("bad so far", bad)
    print("LOADED ALL FILES!")
    with open(name + "_articles.pkl", "wb") as outfile:
        pickle.dump(articles, outfile, pickle.HIGHEST_PROTOCOL)
    print("done dumping")
    dset = fetch_parallel(articles)
    write_data(dset, name)
    
    
def write_data(dset, name):
    print("writing data")
    with open(name + "_source.txt", "a") as file_source:
        with open(name + "_target.txt", "a") as file_target:
#             i = 0
            while not dset.empty():
                article = dset.get()
#                 if not i % 1000: print(i)
#                 print("START", article[1], "END")
                file_source.write(article[0] + "\n")
                file_target.write(article[1] + "\n")
    print("DONE!!!")
                
                            
    
    
    
    

def generate_dset_old(name, condition):
#     with open(name + "_source.txt", "a") as file_source:
#         with open(name + "_target.txt", "a") as file_target:

            # Load data
            for ls in os.listdir('Reuters-full-data-set-master/data')[:5]:
#                 print("New set", ls)
                if ls.endswith('.pkl'):
                    with open('Reuters-full-data-set-master/data/' + ls, 'rb') as f:
                        data = pickle.load(f, encoding='latin1')
                        data = [article for index, article in enumerate(data) if condition(index)][:50]
#                         print("total article count:", len(data))
                        for article in data:
                            title = article["title"]
                            if title[:6] == "TABLE-":
                                continue
                            
                            link = article["href"]
                            start_index = -1
                            try:
                                time1 = time.time()
                                with urllib.request.urlopen(link) as webpage:
                                    time2 = time.time()
                                    content = BeautifulSoup(webpage.read(), "html.parser")
                                    time3 = time.time()
                                    for unnecessary_content in content(["script", "style", "table"]):
                                        unnecessary_content.extract()    
                                    text = content.get_text()
                                    for phrase in start_phrases:
                                        if phrase in text.lower():
                                            phrase_index = text.lower().index(phrase) + len(phrase)
                                            start_index = max(start_index, phrase_index)
                                    end_index = 99999999
                                    for phrase in end_phrases:
                                        if phrase in text.lower():
                                            phrase_index = text.lower().index(phrase)
                                            if text[phrase_index - 1] == "(":
                                                phrase_index -= 1
                                            end_index = min(end_index, phrase_index)
                                    text = text[start_index: end_index]
                                    text = text.replace("\n", " ")
                                    title = title.replace("\n", " ")
                                    print(time3 - time2, time2 - time1)
#                                     file_source.write(title + "\n")
#                                     file_target.write(text + "\n")
#                                     print("suceeded!!!")
                            except Exception as e:
                                pass
#                                 print("BAD ARTICLE!!!")
#                                 print(e)
#                                 print(link)
#     print("DONE!!!")

# Our Standards:The Thomson Reuters Trust Principles

In [ ]:
# 10% val, 20% test, 70% train
print(len(os.listdir('Reuters-full-data-set-master/data')))
# print("Our Standards:The Thomson Reuters Trust Principles".lower())
# print(end_phrases[0])

In [ ]:
# Web scrape -P1
start = time.time()
generate_dset("validation", in_val_set)
end = time.time()
print("Total time: ", end - start)
# generate_dset("train", in_train_set)

In [ ]:
# Web scrape -P1
start = time.time()
generate_dset("test", in_test_set)
end = time.time()
print("Total time: ", end - start)
# generate_dset("train", in_train_set)

In [ ]:
# Web scrape -P1
start = time.time()
generate_dset("train", in_train_set)
end = time.time()
print("Total time: ", end - start)
# generate_dset("train", in_train_set)

In [ ]:
start = time.time()
generate_dset_old("validation", in_val_set)
end = time.time()
print("Total time: ", end - start)

In [ ]:
with open("test_articles.pkl" , "rb") as file:
    articles = pickle.load(file)
print(len(articles))

In [ ]:
import time
import random

from multiprocessing import Process, Queue, current_process, freeze_support

#
# Function run by worker processes
#

def worker(input, output):
    for func, args in iter(input.get, 'STOP'):
        result = calculate(func, args)
        output.put(result)

#
# Function used to calculate result
#

def calculate(func, args):
    result = func(*args)
    return '%s says that %s%s = %s' % \
        (current_process().name, func.__name__, args, result)

#
# Functions referenced by tasks
#

def mul(a, b):
    time.sleep(0.5*random.random())
    return a * b

def plus(a, b):
    time.sleep(0.5*random.random())
    return a + b

#
#
#

def test():
    NUMBER_OF_PROCESSES = 4
    TASKS1 = [(mul, (i, 7)) for i in range(20)]
    TASKS2 = [(plus, (i, 8)) for i in range(10)]

    # Create queues
    task_queue = Queue()
    done_queue = Queue()

    # Submit tasks
    for task in TASKS1:
        task_queue.put(task)

    # Start worker processes
    for i in range(NUMBER_OF_PROCESSES):
        Process(target=worker, args=(task_queue, done_queue)).start()

    # Get and print results
    print('Unordered results:')
    for i in range(len(TASKS1)):
        print('\t', done_queue.get())

    # Add more tasks using `put()`
    for task in TASKS2:
        task_queue.put(task)

    # Get and print some more results
    for i in range(len(TASKS2)):
        print('\t', "lol", done_queue.get())

    # Tell child processes to stop
    for i in range(NUMBER_OF_PROCESSES):
        task_queue.put('STOP')

In [ ]:
test()

In [3]:
## CHANGE THIS ONE!!!

import time
import requests
import random

start_phrases = ["years ago", "min read", "(reuters) - ", "min read*"]
end_phrases = ["our standards:the thomson reuters trust principles", "reporting by", "editing by", "reporting and writing by", "additional reporting", "writing by"]


from multiprocessing import Process, Queue, current_process, freeze_support

def generate_dset(condition):
    # Load data
    articles = []
    for ls in os.listdir('Reuters-full-data-set-master/data'):
#         print("New set", ls)
        if ls.endswith('.pkl'):
            with open('Reuters-full-data-set-master/data/' + ls, 'rb') as f:
                data = pickle.load(f, encoding='latin1')
            new_articles = [article for index, article in enumerate(data) if condition(index)]
            articles += new_articles
    print("LOADED ALL FILES!")
    return articles
    

#
# Function run by worker processes
#
def read_webpage(article):
    title = article["title"]
    if title[:6] == "TABLE-":
        return None, "TABLE"

    link = article["href"]
#     print("LINK", type(link))
    start_index = -1
    try:
#         title = "nice try"
#         text = "nicer try"
#         with urllib.request.urlopen(link) as webpage:
#             pass
#         with open("testing_again.csv", "w") as file:
#             pass
#         url_reader = urllib.request.urlopen(link)
#         webpage = url_reader.read()
#         url_reader.close()
#         time.sleep(3)
        
#         with urllib.request.urlopen(link) as webpage:
#             page = webpage.read()
        
        page = requests.get(link).content
            
        content = BeautifulSoup(page, "html.parser")
        for unnecessary_content in content(["script", "style", "table"]):
            unnecessary_content.extract()    
        text = content.get_text()
        for phrase in start_phrases:
            if phrase in text.lower():
                phrase_index = text.lower().index(phrase) + len(phrase)
                start_index = max(start_index, phrase_index)
        end_index = 99999999
        for phrase in end_phrases:
            if phrase in text.lower():
                phrase_index = text.lower().index(phrase)
                if text[phrase_index - 1] == "(":
                    phrase_index -= 1
                end_index = min(end_index, phrase_index)
        text = text[start_index: end_index]
        text = text.replace("\n", " ")
        title = title.replace("\n", " ")
#         print(title)
#         print(" ")
#         print(text)
#         print(" ")
#         print("")
        
        
        return str(title), str(text)
    except Exception as e:
        return None, e, link

def worker(input, output):
    for article in iter(input.get, 'STOP'):
        result = read_webpage(article)
        output.put(result)

def run_all(name, condition):
    print("starting")
    start_time = time.time()
    NUMBER_OF_PROCESSES = 150
    all_tasks = generate_dset(condition)
    print("Tasks", len(all_tasks))
    iteration = 0
    
    while iteration < len(all_tasks):
        print("ITERATION!!!", iteration)
        start = iteration
        end = min(start + 10000, len(all_tasks))
        tasks = all_tasks[start:end]
        iteration = end

        # Create queues
        task_queue = Queue()
        done_queue = Queue()

        # Start worker processes
        for i in range(NUMBER_OF_PROCESSES):
            Process(target=worker, args=(task_queue, done_queue)).start()

        n = len(tasks)
        # Submit tasks
        i = 0
        for task in tasks:
            if not i % 1000:
                print("loading task %d" % i)
            task_queue.put(task)
            i += 1

        print("Loaded everything into the queue!")

#         # Start worker processes
#         for i in range(NUMBER_OF_PROCESSES):
#             Process(target=worker, args=(task_queue, done_queue)).start()

        print("starting processes")

        # Get and print results
        print("writing data")
        with open(name + "_source.txt", "a") as file_source:
            with open(name + "_target.txt", "a") as file_target:
                for i in range(len(tasks)):
                    if not i % 100:
                        print("current progress: %d of %d" % (i, n))
                    article = done_queue.get()
                    if article[0] is not None:
    #                     print("writing", article[0])
                        file_source.write(article[0] + "\n")
                        file_target.write(article[1] + "\n")
                    else:
                        pass
                        print("WHAT'S WRONG?", article[1])
        print("DONE!!!")

        # Tell child processes to stop
        for i in range(NUMBER_OF_PROCESSES):
            task_queue.put('STOP')
        end_time = time.time()
        print("total time", end_time - start_time)
        
#kangaroo
print("loaded")

loaded


In [ ]:
run_all("val", in_val_set) #CHANGE TITLE!!!

starting
LOADED ALL FILES!
Tasks 856725
ITERATION!!! 0
loading task 0
loading task 1000
loading task 2000
loading task 3000
loading task 4000
loading task 5000
loading task 6000
loading task 7000
loading task 8000
loading task 9000
Loaded everything into the queue!
starting processes
writing data
current progress: 0 of 10000
current progress: 100 of 10000
current progress: 200 of 10000
current progress: 300 of 10000
WHAT'S WRONG? TABLE
current progress: 400 of 10000
current progress: 500 of 10000
current progress: 600 of 10000
current progress: 700 of 10000
current progress: 800 of 10000
WHAT'S WRONG? TABLE
current progress: 900 of 10000
current progress: 1000 of 10000
current progress: 1100 of 10000
current progress: 1200 of 10000
current progress: 1300 of 10000
WHAT'S WRONG? TABLE
WHAT'S WRONG? TABLE
current progress: 1400 of 10000
current progress: 1500 of 10000
current progress: 1600 of 10000
current progress: 1700 of 10000
current progress: 1800 of 10000
current progress: 1900 of 

WHAT'S WRONG? TABLE
current progress: 2600 of 10000
WHAT'S WRONG? TABLE
WHAT'S WRONG? TABLE
WHAT'S WRONG? TABLE
WHAT'S WRONG? TABLE
WHAT'S WRONG? TABLE
WHAT'S WRONG? TABLE
WHAT'S WRONG? TABLE
WHAT'S WRONG? TABLE
WHAT'S WRONG? TABLE
WHAT'S WRONG? TABLE
WHAT'S WRONG? TABLE
WHAT'S WRONG? TABLE
WHAT'S WRONG? TABLE
WHAT'S WRONG? TABLE
WHAT'S WRONG? TABLE
WHAT'S WRONG? TABLE
WHAT'S WRONG? TABLE
WHAT'S WRONG? TABLE
current progress: 2700 of 10000
WHAT'S WRONG? TABLE
WHAT'S WRONG? TABLE
WHAT'S WRONG? TABLE
WHAT'S WRONG? TABLE
WHAT'S WRONG? TABLE
WHAT'S WRONG? TABLE
WHAT'S WRONG? TABLE
WHAT'S WRONG? TABLE
WHAT'S WRONG? TABLE
WHAT'S WRONG? TABLE
current progress: 2800 of 10000
current progress: 2900 of 10000
WHAT'S WRONG? TABLE
current progress: 3000 of 10000
current progress: 3100 of 10000
WHAT'S WRONG? TABLE
current progress: 3200 of 10000
current progress: 3300 of 10000
current progress: 3400 of 10000
WHAT'S WRONG? TABLE
WHAT'S WRONG? TABLE
current progress: 3500 of 10000
current progress: 36

current progress: 3400 of 10000
current progress: 3500 of 10000
current progress: 3600 of 10000
current progress: 3700 of 10000
current progress: 3800 of 10000
WHAT'S WRONG? ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response',))
current progress: 3900 of 10000
WHAT'S WRONG? None: Max retries exceeded with url: /article/2013/02/06/bc-macarthur-foundation-idUSnPn2063486+100+PRN20130206 (Caused by None)
current progress: 4000 of 10000
current progress: 4100 of 10000
current progress: 4200 of 10000
WHAT'S WRONG? TABLE
WHAT'S WRONG? TABLE
current progress: 4300 of 10000
current progress: 4400 of 10000
current progress: 4500 of 10000
WHAT'S WRONG? TABLE
WHAT'S WRONG? TABLE
WHAT'S WRONG? TABLE
WHAT'S WRONG? TABLE
WHAT'S WRONG? TABLE
WHAT'S WRONG? TABLE
current progress: 4600 of 10000
current progress: 4700 of 10000
WHAT'S WRONG? None: Max retries exceeded with url: /article/2014/09/10/pa-hartford-funds-idUSnBw105721a+100+BSW20140910 (Caused by None)
curr

WHAT'S WRONG? TABLE
WHAT'S WRONG? TABLE
WHAT'S WRONG? TABLE
WHAT'S WRONG? TABLE
WHAT'S WRONG? TABLE
WHAT'S WRONG? TABLE
WHAT'S WRONG? TABLE
WHAT'S WRONG? TABLE
WHAT'S WRONG? TABLE
WHAT'S WRONG? TABLE
WHAT'S WRONG? TABLE
WHAT'S WRONG? TABLE
WHAT'S WRONG? TABLE
WHAT'S WRONG? TABLE
WHAT'S WRONG? TABLE
WHAT'S WRONG? TABLE
WHAT'S WRONG? TABLE
WHAT'S WRONG? TABLE
WHAT'S WRONG? ('Connection aborted.', ConnectionResetError(54, 'Connection reset by peer'))
WHAT'S WRONG? ('Connection aborted.', ConnectionResetError(54, 'Connection reset by peer'))
WHAT'S WRONG? ('Connection aborted.', ConnectionResetError(54, 'Connection reset by peer'))
current progress: 9100 of 10000
WHAT'S WRONG? ("Connection broken: ConnectionResetError(54, 'Connection reset by peer')", ConnectionResetError(54, 'Connection reset by peer'))
WHAT'S WRONG? ("Connection broken: ConnectionResetError(54, 'Connection reset by peer')", ConnectionResetError(54, 'Connection reset by peer'))
WHAT'S WRONG? TABLE
WHAT'S WRONG? TABLE
curr

current progress: 3800 of 10000
current progress: 3900 of 10000
WHAT'S WRONG? TABLE
current progress: 4000 of 10000
current progress: 4100 of 10000
current progress: 4200 of 10000
WHAT'S WRONG? TABLE
current progress: 4300 of 10000
current progress: 4400 of 10000
current progress: 4500 of 10000
current progress: 4600 of 10000
current progress: 4700 of 10000
current progress: 4800 of 10000
WHAT'S WRONG? TABLE
current progress: 4900 of 10000
WHAT'S WRONG? TABLE
current progress: 5000 of 10000
current progress: 5100 of 10000
current progress: 5200 of 10000
current progress: 5300 of 10000
current progress: 5400 of 10000
current progress: 5500 of 10000
current progress: 5600 of 10000
current progress: 5700 of 10000
WHAT'S WRONG? TABLE
current progress: 5800 of 10000
WHAT'S WRONG? TABLE
WHAT'S WRONG? TABLE
current progress: 5900 of 10000
current progress: 6000 of 10000
current progress: 6100 of 10000
current progress: 6200 of 10000
current progress: 6300 of 10000
WHAT'S WRONG? TABLE
WHAT'S W

current progress: 9700 of 10000
current progress: 9800 of 10000
current progress: 9900 of 10000
DONE!!!
total time 2581.920999288559
ITERATION!!! 60000
loading task 0
loading task 1000
loading task 2000
loading task 3000
loading task 4000
loading task 5000
loading task 6000
loading task 7000
loading task 8000
loading task 9000
Loaded everything into the queue!
starting processes
writing data
current progress: 0 of 10000
WHAT'S WRONG? TABLE
current progress: 100 of 10000
current progress: 200 of 10000
WHAT'S WRONG? TABLE
current progress: 300 of 10000
WHAT'S WRONG? TABLE
current progress: 400 of 10000
current progress: 500 of 10000
current progress: 600 of 10000
WHAT'S WRONG? TABLE
current progress: 700 of 10000
current progress: 800 of 10000
WHAT'S WRONG? None: Max retries exceeded with url: /article/2012/06/10/us-france-election-socialist-idUSBRE8590C920120610 (Caused by None)
WHAT'S WRONG? None: Max retries exceeded with url: /article/2012/06/10/us-australia-miners-idUSBRE8590FC20120

WHAT'S WRONG? TABLE
WHAT'S WRONG? TABLE
WHAT'S WRONG? TABLE
WHAT'S WRONG? TABLE
WHAT'S WRONG? TABLE
WHAT'S WRONG? TABLE
WHAT'S WRONG? TABLE
WHAT'S WRONG? TABLE
WHAT'S WRONG? TABLE
WHAT'S WRONG? TABLE
WHAT'S WRONG? TABLE
current progress: 900 of 10000
WHAT'S WRONG? TABLE
current progress: 1000 of 10000
current progress: 1100 of 10000
WHAT'S WRONG? TABLE
WHAT'S WRONG? TABLE
WHAT'S WRONG? TABLE
WHAT'S WRONG? TABLE
WHAT'S WRONG? TABLE
current progress: 1200 of 10000
current progress: 1300 of 10000
WHAT'S WRONG? TABLE
current progress: 1400 of 10000
current progress: 1500 of 10000
WHAT'S WRONG? TABLE
WHAT'S WRONG? TABLE
WHAT'S WRONG? TABLE
WHAT'S WRONG? TABLE
WHAT'S WRONG? TABLE
WHAT'S WRONG? TABLE
WHAT'S WRONG? TABLE
WHAT'S WRONG? TABLE
WHAT'S WRONG? TABLE
WHAT'S WRONG? TABLE
WHAT'S WRONG? TABLE
WHAT'S WRONG? TABLE
WHAT'S WRONG? TABLE
WHAT'S WRONG? TABLE
WHAT'S WRONG? TABLE
WHAT'S WRONG? TABLE
WHAT'S WRONG? TABLE
current progress: 1600 of 10000
WHAT'S WRONG? TABLE
WHAT'S WRONG? TABLE
WHAT'

In [ ]:
run_all("test", in_test_set)

In [ ]:
run_all("train", in_train_set)

In [ ]:
150, bottom total time 24.13818097114563
150, top: 24.01080012321472
8, top; total time 107.93835997581482
100, top; total time 63.626007080078125
149: total time 58.94413423538208

In [ ]:
num_workers = multiprocessing.cpu_count()
print(num_workers)

In [ ]:
print(len(os.listdir('Reuters-full-data-set-master/data')))

In [ ]:
def please_work(name, condition):
    name = "val"

    start = time.time()
    print("generating")
    urls = generate_dset(condition)
    print("got %d urls" % len(urls))

    import gevent
    from gevent import monkey

    # patches stdlib (including socket and ssl modules) to cooperate with other greenlets
    monkey.patch_all()


    print("about to get real")

    # def print_head(url):
    #     print ('Starting %s' % url)
    #     data = urllib2.urlopen(url).read()
    #     print ('%s: %s bytes: %r' % (url, len(data), data[:50]))

    middle = time.time()

    jobs = [gevent.spawn(read_webpage, url) for url in urls]

    print("made jobs")

    gevent.joinall(jobs)

    i = 0
    with open(name + "_source.txt", "a") as file_source:
        with open(name + "_target.txt", "a") as file_target:
            for article in jobs:
                if article.ready():
                    if article.value[0] is not None:
                        i += 1
                        file_source.write(article.value[0] + "\n")
                        file_target.write(article.value[1] + "\n")
                    else:
                        print(article.value[1])
                        print(article.value[2])
                else:
                    print("not ready")
                    
    end = time.time()
    print("total count: %d" % i)
    print("intermediate time ", end - middle)
    print("done in ", end - start)

In [ ]:
please_work("val", in_val_set)

In [ ]:
import multiprocessing
import time
import psutil

# class ActivePool(object):
#     def __init__(self):
#         super(ActivePool, self).__init__()
#         self.mgr = multiprocessing.Manager()
#         self.active = self.mgr.list()
#         self.lock = multiprocessing.Lock()

#     def make_active(self, name):
#         with self.lock:
#             self.active.append(name)

#     def make_inactive(self, name):
#         with self.lock:
#             self.active.remove(name)

#     def __str__(self):
#         with self.lock:
#             return str(self.active)

    
class DsetMaker:
    """ Builds Reuters dataset """
    def __init__(self):
        self.start_phrases = ["years ago", "min read", "(reuters) - ", "min read*"]
        self.end_phrases = ["our standards:the thomson reuters trust principles", "reporting by", "editing by", "reporting and writing by", "additional reporting", "writing by"]
            
    def read_webpage(self, s, pool, results_queue, article):
#         proc = psutil.Process()
#         print("SECOND", proc.open_files())
        process_name = multiprocessing.current_process().name
        with s:
#             pool.make_active(process_name)
            title = article["title"]
            if title[:6] == "TABLE-":
                return

            link = article["href"]
            start_index = -1
            try:
                time.sleep(.5)
                results_queue.put(("aaa", "BBB"))
#                 with urllib.request.urlopen(link) as webpage:
#                     page = webpage.read()
#                 content = BeautifulSoup(page, "html.parser")
#                 for unnecessary_content in content(["script", "style", "table"]):
#                     unnecessary_content.extract()    
#                 text = content.get_text()
#                 for phrase in start_phrases:
#                     if phrase in text.lower():
#                         phrase_index = text.lower().index(phrase) + len(phrase)
#                         start_index = max(start_index, phrase_index)
#                 end_index = 99999999
#                 for phrase in end_phrases:
#                     if phrase in text.lower():
#                         phrase_index = text.lower().index(phrase)
#                         if text[phrase_index - 1] == "(":
#                             phrase_index -= 1
#                         end_index = min(end_index, phrase_index)
#                 text = text[start_index: end_index]
#                 text = text.replace("\n", " ")
#                 title = title.replace("\n", " ")
#                 results_queue.put((title, text))
            except:
                results_queue.put(None)
#             pool.make_inactive(process_name)
    
    def generate_dset(self, name, condition):
        # Load data
        articles = []
        for ls in os.listdir('Reuters-full-data-set-master/data')[:5]:
            print("New set", ls)
            if ls.endswith('.pkl'):
                with open('Reuters-full-data-set-master/data/' + ls, 'rb') as f:
                    data = pickle.load(f, encoding='latin1')
                    new_articles = [article for index, article in enumerate(data) if condition(index)][:50]
                    articles += new_articles
                    print("total article count:", len(articles))
        print("LOADED ALL FILES!")
        with open(name + "_articles.pkl", "wb") as outfile:
            pickle.dump(articles, outfile, pickle.HIGHEST_PROTOCOL)
        print("done dumping")
        self.build_dset(articles, name)

    def build_dset(self, articles, name):
        """
        Building dset
        """
        with multiprocessing.Pool(nprocess) as pool:
        
#         pool = ActivePool()
        pool = None
        s = multiprocessing.Semaphore(50)
#         results_queue = multiprocessing.Queue()
        results_queue = Queue()
        jobs = [
            multiprocessing.Process(
                target = self.read_webpage,
                name = article["href"],
                args = (
                    s,
                    pool,
                    results_queue,
                    article
                )
            )
            for article in articles
        ]
        load_start = time.time()
        proc = psutil.Process()
        print("FIRST TIME")
        print(proc.open_files())
        [j.start() for j in jobs]# yep

        # pull results from queue before joining threads to avoid pipe deadlock.
        # https://tinyurl.com/yda2aa6k
#         with open(name + "_source.txt", "a") as file_source:
#             with open(name + "_target.txt", "a") as file_target:
#                 pulled_results = 0
#                 print("writing data")
#                 write_time = time.time()
#                 while(pulled_results < len(articles)):
#                     try:
#                         while(not results_queue.empty()):
#                             article = results_queue.get()
#                             pulled_results += 1
#                             if not article is None:
#                                 pass
#         #                         file_source.write(article[0] + "\n")
#         #                         file_target.write(article[1] + "\n")
#                             print("DONE!!!")
#                     except Queue.empty:
#                         pass
#                     print("sleeping")
#                     time.sleep(1)
#                 after_write = time.time()
#                 print("writing (and waiting for things to write) took", after_write - write_time)

        [j.join() for j in jobs] 
        load_end = time.time()
        print("Load time is ", load_end - load_start)
    
#         with open(name + "_source.txt", "a") as file_source:
#             with open(name + "_target.txt", "a") as file_target:
#                 file_source.write("HI1")
#                 file_target.write("BYE!")
#                 print("writing data")
#         write_end = time.time()
#         print("writing took", write_end - load_end)
                
#                 pulled_results = 0
#                 print("writing data")
#                 write_time = time.time()
#                 while(pulled_results < len(articles)):
#                     try:
#                         while(not results_queue.empty()):
#                             article = results_queue.get()
#                             pulled_results += 1
#                             if not article is None:
#                                 pass
#                                 file_source.write(article[0] + "\n")
#                                 file_target.write(article[1] + "\n")
#                             print("DONE!!!")
#                     except Queue.empty:
#                         pass
#                     print("sleeping")
#                     time.sleep(1)
#                 after_write = time.time()
#                 print("writing (and waiting for things to write) took", after_write - write_time)
        
        print("ALL DONE!!!")

print("Done")

In [ ]:
overall_start = time.time()
d = DsetMaker()
d.generate_dset("isthisworking", in_val_set)
overall_end = time.time()
print("Overall Time: ", overall_end - overall_start)

In [ ]:
# DON'T CHANGE THIS!!!


import multiprocessing
import time

from activePool import ActivePool
from celex import Celex
from config import GAConfig

class ActivePool(object):
    def __init__(self):
        super(ActivePool, self).__init__()
        self.mgr = multiprocessing.Manager()
        self.active = self.mgr.list()
        self.lock = multiprocessing.Lock()

    def make_active(self, name):
        with self.lock:
            self.active.append(name)

    def make_inactive(self, name):
        with self.lock:
            self.active.remove(name)

    def __str__(self):
        with self.lock:
            return str(self.active)

class ComputeFitness:
    """ Computes the fitness of chromosomes in a population concurrently """

    def __init__(self):
        self.celex = Celex()#B

    def compute(self, population):
        """
        Compute the fitness of all chromosomes in the population.
        Updates the fitness value of all chromosomes.
        Chromosome fitness calculation is done in separate processes.
        """
        sizes = (GAConfig["training_size_hmm"], GAConfig["testing_size_hmm"])#B
        self.celex.load_sets(sizes[0], sizes[1])#B

        pool = ActivePool()
        s = multiprocessing.Semaphore(GAConfig["max_thread_count"])#MAYBE
        results_queue = multiprocessing.Queue(len(population) + 1)#yep!
        jobs = [ #yep... read here
            multiprocessing.Process(
                target = self._compute_single_fitness,
                name = str(i),
                args = (
                    i,
                    s,
                    pool,
                    results_queue,
                    population[i].get_genes()
                )
            )
            for i in range(len(population))
        ]
        [j.start() for j in jobs]# yep

        # pull results from queue before joining threads to avoid pipe deadlock.
        # https://tinyurl.com/yda2aa6k
        pulled_results = 0
        while(pulled_results < len(population)): # while not done
            try:
                while(not results_queue.empty()): # Write to file here
                    result = results_queue.get()
                    index = result[0]
                    fitness = result[1]
                    results_list = result[2]
                    population[index].set_fitness(fitness)
                    population[index].set_results(results_list)
                    pulled_results += 1

            except Queue.empty:
                pass
            time.sleep(1)

        [j.join() for j in jobs] # yep
        return population

    def _compute_single_fitness(self, i, s, pool, results_queue, genes):
        """
        Calculates and puts updated fitness on the results_queue.
        Args:
            i (int): process and population index
            s (multiprocessing.Semaphore)
            pool (ActivePool): manager of the pool and locks
            results_queue (multiprocessing.Queue): communication
                        between processes.
            genes (list<list<char>>): genes of a Chromosome
        """
        process_name = multiprocessing.current_process().name
        with s:
            pool.make_active(process_name)
            ps_model = self.celex.train_hmm(genes)
            fitness, all_results = self.celex.test_hmm(ps_model)
            ps_model = None
            results_queue.put((i, fitness, all_results))
            pool.make_inactive(process_name)


In [ ]:
url1 = "http://www.reuters.com/article/companyNewsAndPR/idUSTP13157220070102"
url2 = "http://www.reuters.com/article/2011/09/09/indian-stocks-open-idUSI8E7J302G20110909"
url3 = "http://www.reuters.com/article/companyNewsAndPR/idUSL1136380220070111"
url4 = "http://www.reuters.com/article/india-cenbank-refinance-idUSB8N12101C20151015"
tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
start_phrase = "years ago"
end_phrase = "Our Standards:The Thomson Reuters Trust Principles."

with urllib.request.urlopen(url3) as webpage:
    content = BeautifulSoup(webpage.read(), "html.parser")
    for script in content(["script", "style"]):
        script.extract()
    print(content)
#     text = content.get_text()
#     start_index = text.index(start_phrase) + len(start_phrase)
#     end_index = text.index(end_phrase)
#     text = text[start_index: end_index]
#     lines = (line.strip() for line in text.splitlines() if not line.strip() == "")
#     for i, line in enumerate(lines):
# #         print("Line " + str(i) + ": " + line)
#         tokens = tokenizer.tokenize(line)
#         for token in tokens:
#             print("\n")
#             print(token)
# #         print(tokens)
        


In [ ]:
import multiprocessing
import time
import psutil

class ActivePool(object):
    def __init__(self):
        super(ActivePool, self).__init__()
        self.mgr = multiprocessing.Manager()
        self.active = self.mgr.list()
        self.lock = multiprocessing.Lock()

    def make_active(self, name):
        with self.lock:
            self.active.append(name)

    def make_inactive(self, name):
        with self.lock:
            self.active.remove(name)

    def __str__(self):
        with self.lock:
            return str(self.active)

    
class DsetMaker:
    """ Builds Reuters dataset """
    def __init__(self):
        self.start_phrases = ["years ago", "min read", "(reuters) - ", "min read*"]
        self.end_phrases = ["our standards:the thomson reuters trust principles", "reporting by", "editing by", "reporting and writing by", "additional reporting", "writing by"]
            
    def read_webpage(self, s, pool, results_queue, article):
        proc = psutil.Process()
        print("SECOND", proc.open_files())
        process_name = multiprocessing.current_process().name
        with s:
            pool.make_active(process_name)
            title = article["title"]
            if title[:6] == "TABLE-":
                return

            link = article["href"]
            start_index = -1
            try:
                time.sleep(.5)
                results_queue.put(("aaa", "BBB"))
#                 with urllib.request.urlopen(link) as webpage:
#                     page = webpage.read()
#                 content = BeautifulSoup(page, "html.parser")
#                 for unnecessary_content in content(["script", "style", "table"]):
#                     unnecessary_content.extract()    
#                 text = content.get_text()
#                 for phrase in start_phrases:
#                     if phrase in text.lower():
#                         phrase_index = text.lower().index(phrase) + len(phrase)
#                         start_index = max(start_index, phrase_index)
#                 end_index = 99999999
#                 for phrase in end_phrases:
#                     if phrase in text.lower():
#                         phrase_index = text.lower().index(phrase)
#                         if text[phrase_index - 1] == "(":
#                             phrase_index -= 1
#                         end_index = min(end_index, phrase_index)
#                 text = text[start_index: end_index]
#                 text = text.replace("\n", " ")
#                 title = title.replace("\n", " ")
#                 results_queue.put((title, text))
            except:
                results_queue.put(None)
            pool.make_inactive(process_name)
    
    def generate_dset(self, name, condition):
        # Load data
        articles = []
        for ls in os.listdir('Reuters-full-data-set-master/data')[:5]:
            print("New set", ls)
            if ls.endswith('.pkl'):
                with open('Reuters-full-data-set-master/data/' + ls, 'rb') as f:
                    data = pickle.load(f, encoding='latin1')
                    new_articles = [article for index, article in enumerate(data) if condition(index)][:50]
                    articles += new_articles
                    print("total article count:", len(articles))
        print("LOADED ALL FILES!")
        with open(name + "_articles.pkl", "wb") as outfile:
            pickle.dump(articles, outfile, pickle.HIGHEST_PROTOCOL)
        print("done dumping")
        self.build_dset(articles, name)

    def build_dset(self, articles, name):
        """
        Building dset
        """
        
        pool = ActivePool()
        s = multiprocessing.Semaphore(50)
#         results_queue = multiprocessing.Queue()
        results_queue = Queue()
        jobs = [
            multiprocessing.Process(
                target = self.read_webpage,
                name = article["href"],
                args = (
                    s,
                    pool,
                    results_queue,
                    article
                )
            )
            for article in articles
        ]
        load_start = time.time()
        proc = psutil.Process()
        print("FIRST TIME")
        print(proc.open_files())
        [j.start() for j in jobs]# yep

        # pull results from queue before joining threads to avoid pipe deadlock.
        # https://tinyurl.com/yda2aa6k
#         with open(name + "_source.txt", "a") as file_source:
#             with open(name + "_target.txt", "a") as file_target:
#                 pulled_results = 0
#                 print("writing data")
#                 write_time = time.time()
#                 while(pulled_results < len(articles)):
#                     try:
#                         while(not results_queue.empty()):
#                             article = results_queue.get()
#                             pulled_results += 1
#                             if not article is None:
#                                 pass
#         #                         file_source.write(article[0] + "\n")
#         #                         file_target.write(article[1] + "\n")
#                             print("DONE!!!")
#                     except Queue.empty:
#                         pass
#                     print("sleeping")
#                     time.sleep(1)
#                 after_write = time.time()
#                 print("writing (and waiting for things to write) took", after_write - write_time)

        [j.join() for j in jobs] 
        load_end = time.time()
        print("Load time is ", load_end - load_start)
    
#         with open(name + "_source.txt", "a") as file_source:
#             with open(name + "_target.txt", "a") as file_target:
#                 file_source.write("HI1")
#                 file_target.write("BYE!")
#                 print("writing data")
#         write_end = time.time()
#         print("writing took", write_end - load_end)
                
#                 pulled_results = 0
#                 print("writing data")
#                 write_time = time.time()
#                 while(pulled_results < len(articles)):
#                     try:
#                         while(not results_queue.empty()):
#                             article = results_queue.get()
#                             pulled_results += 1
#                             if not article is None:
#                                 pass
#                                 file_source.write(article[0] + "\n")
#                                 file_target.write(article[1] + "\n")
#                             print("DONE!!!")
#                     except Queue.empty:
#                         pass
#                     print("sleeping")
#                     time.sleep(1)
#                 after_write = time.time()
#                 print("writing (and waiting for things to write) took", after_write - write_time)
        
        print("ALL DONE!!!")

print("Done")